In [1]:
import ants
import numpy as np
import pandas as pd
import os
from utils.asegStats_to_df import asegStats_to_df
from utils.texturemap import *

In [2]:


clinical_df = pd.read_csv('data/summary/ONDRI_summary.csv')
clinical_df = clinical_df[clinical_df['COHORT']!= 'VCI']
clinical_df = clinical_df[~pd.isna(clinical_df['NII_FILENAME_T1'])]

In [11]:
segfiles = []
for i, row in clinical_df.iterrows():
    print(i)
    t1_segfile_path = 'outputs/fastsurfer/scratch/%s/stats/aseg+DKT.stats' % row['NII_FILENAME_T1'].replace('.nii.gz', '')
    t1_mask_path = 'outputs/fastsurfer/scratch/%s/mri/mask.mgz' % row['NII_FILENAME_T1'].replace('.nii.gz', '')
    if os.path.exists(t1_segfile_path):
        segfile = asegStats_to_df(t1_segfile_path)
        segfile = segfile[['StructName', 'Volume_mm3']]
        segfile = segfile.pivot(columns='StructName', index=[])
        segfile['SUBJECT'] = row['SUBJECT']
        segfile['total_intracranial_volume'] = ants.image_read(t1_mask_path).sum()

        segfiles.append(segfile)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
21
22
23
24
25
26
27
28
29
31
32
33
34
35
36
37
38
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
170
171
173
174
175
176
177
178
179
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
320
321
322
323
324
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347


In [31]:
segfiles_df = []
for segfile in segfiles:
    df = pd.DataFrame(segfile).transpose()
    df.columns = df.columns.get_level_values(1)[:-2].tolist() + ['SUBJECT', 'total_intracranial_volume']
    segfiles_df.append(df)

                           Volume_mm3                         \
StructName Left-Cerebral-White-Matter Left-Lateral-Ventricle   
0                          193817.173                25280.3   

                                                           \
StructName Left-Inf-Lat-Vent Left-Cerebellum-White-Matter   
0                    1878.22                    11167.041   

                                                                           \
StructName Left-Cerebellum-Cortex Left-Thalamus Left-Caudate Left-Putamen   
0                       46706.955       6554.43     3980.082      4888.28   

                                        ...                                  \
StructName Left-Pallidum 3rd-Ventricle  ... ctx-rh-rostralanteriorcingulate   
0               2094.421      2370.015  ...                        2261.507   

                                                               \
StructName ctx-rh-rostralmiddlefrontal ctx-rh-superiorfrontal   
0                      

In [37]:
output_df = pd.concat(segfiles_df)
output_df = output_df.drop(columns=['CC_Posterior', 'CC_Mid_Posterior', 'CC_Central', 'CC_Mid_Anterior', 'CC_Anterior'])
output_df.to_csv('outputs/fs_t1_volumes.csv', index=False)